# Antimatter rocket

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lookbusy1344/Relativity/blob/main/Python/Antimatter%20rocket.ipynb)

This repository is primarily focused on the purely kinematic equations of Special Relativity. However, it also includes some basic equations for an antimatter rocket, in both Python and Typescript. These are used in the interactively in web calculator:

https://lookbusy1344.github.io/Relativity/

The Python propulsion equations are in https://github.com/lookbusy1344/Relativity/blob/main/Python/propulsion.py

## Two kinds of antimatter rocket

There are two basic kinds of antimatter rocket:

1. Photonic rocket: This uses the energy from matter-antimatter annihilation to produce photons (gamma rays) which are emitted out the back of the rocket to produce thrust. This is the simplest kind of antimatter rocket, but is unrealistic because it is very difficult to collimate the gamma rays into a beam.

2. Charged-pion rocket: This uses the charged pions produced in proton-antiproton annihilation to produce thrust. The charged pions can be collimated using magnetic fields (you can make the thrust go in a particular direction), making this a more realistic design for an antimatter rocket.

## We use the charged-pion rocket equations here

Obviously this technology is beyond our current capabilities, but is scientifically plausible. We assume various levels of nozzle efficiency to account for engineering challenges. The equations are in propulsion.py, and we use them here to calculate acceleration time and propellant fraction for a charged-pion antimatter rocket.

85% nozzle efficiency (~57% total system efficiency) is probably the maximum achievable.

In [13]:
# Download propulsion.py from your GitHub repo if not present. Important for colab.
import os

if not os.path.exists("relativity_lib.py"):
    !wget -q https://raw.githubusercontent.com/lookbusy1344/Relativity/main/Python/relativity_lib.py

if not os.path.exists("propulsion.py"):
    !wget -q https://raw.githubusercontent.com/lookbusy1344/Relativity/main/Python/propulsion.py

import mpmath as mp
import relativity_lib as rl
import propulsion as prop
from prettytable import PrettyTable, TableStyle
from IPython.display import Markdown

# Configure relativity_lib for precision calculations
rl.configure(50)

fuel_mass = 1000.0  # kg, half is matter, half is antimatter
dry_mass = 500.0  # kg, the spacecraft dry mass

t = prop.pion_rocket_accel_time(fuel_mass, dry_mass, nozzle_efficiency=0.85)
years = t / rl.seconds_per_year
print(
    f"Dry mass {float(dry_mass):.0f}, fuel mass {float(fuel_mass):.0f} means {float(years):.2f} years at 1g "
    f"(85% nozzle efficiency, ~57% total system efficiency)"
)
print()

# Calculate propellant fraction for 3.52 years of 1g acceleration at 85% nozzle efficiency
years = rl.ensure("3.52")
secs = rl.seconds_per_year * years
f = prop.pion_rocket_fuel_fraction(secs, None, 0.85)
print(
    f"Propellant mass fraction: {float(f) * 100:.4f}% for {float(years)} years of 1g "
    f"(85% nozzle efficiency, ~57% total system efficiency)"
)

table = PrettyTable(
    [
        "Nozzle efficiency",
        "Proper time (years)",
        "System efficiency",
        "Propellant fraction (%)",
    ]
)
table.set_style(TableStyle.MARKDOWN)
table.align = "r"  # right align everything

perc = mp.mpf("100")

# Define efficiencies and time periods
nozzle_efficiencies = [0.70, 0.75, 0.80, 0.85]
proper_times_years = [0.01, 0.1, 1, 2, 3, 4, 6, 8, 10, 15, 20, 25]

# Populate table with combinations
for eff in nozzle_efficiencies:
    for years in proper_times_years:
        secs = rl.seconds_per_year * rl.ensure(str(years))
        f = prop.pion_rocket_fuel_fraction(secs, None, eff)
        system_eff = eff * 0.67  # approximate system efficiency
        frac = rl.format_mpf_significant(f * perc, 2, "9")

        table.add_row([f"{eff:.0%}", years, f"{system_eff:.0%}", frac])

print()
print(
    "Here we show propellant fractions (how much of the ship needs to be fuel) for various nozzle efficiencies and proper times at 1g acceleration"
)
print(
    "Nozzle efficiencies range from 70% to 85%, and proper times from 1 year to 25 years"
)
markdown_table = table.get_string()
display(Markdown(markdown_table));

Dry mass 500, fuel mass 1000 means 0.57 years at 1g (85% nozzle efficiency, ~57% total system efficiency)

Propellant mass fraction: 99.8910% for 3.52 years of 1g (85% nozzle efficiency, ~57% total system efficiency)

Here we show propellant fractions (how much of the ship needs to be fuel) for various nozzle efficiencies and proper times at 1g acceleration
Nozzle efficiencies range from 70% to 85%, and proper times from 1 year to 25 years


| Nozzle efficiency | Proper time (years) | System efficiency |      Propellant fraction (%) |
|-----------------: |-------------------: |-----------------: |----------------------------: |
|               70% |                0.01 |               47% |                         2.32 |
|               70% |                 0.1 |               47% |                       20.968 |
|               70% |                   1 |               47% |                        90.49 |
|               70% |                   2 |               47% |                        99.09 |
|               70% |                   3 |               47% |                       99.914 |
|               70% |                   4 |               47% |                      99.9918 |
|               70% |                   6 |               47% |                    99.999926 |
|               70% |                   8 |               47% |                  99.99999933 |
|               70% |                  10 |               47% |                99.9999999939 |
|               70% |                  15 |               47% |           99.999999999999953 |
|               70% |                  20 |               47% |      99.99999999999999999963 |
|               70% |                  25 |               47% | 99.9999999999999999999999971 |
|               75% |                0.01 |               50% |                         2.17 |
|               75% |                 0.1 |               50% |                        19.71 |
|               75% |                   1 |               50% |                        88.87 |
|               75% |                   2 |               50% |                        98.76 |
|               75% |                   3 |               50% |                        99.86 |
|               75% |                   4 |               50% |                       99.984 |
|               75% |                   6 |               50% |                     99.99981 |
|               75% |                   8 |               50% |                   99.9999976 |
|               75% |                  10 |               50% |                 99.999999971 |
|               75% |                  15 |               50% |            99.99999999999950 |
|               75% |                  20 |               50% |       99.9999999999999999916 |
|               75% |                  25 |               50% |   99.99999999999999999999985 |
|               80% |                0.01 |               54% |                         2.03 |
|               80% |                 0.1 |               54% |                        18.60 |
|               80% |                   1 |               54% |                        87.24 |
|               80% |                   2 |               54% |                        98.37 |
|               80% |                   3 |               54% |                        99.79 |
|               80% |                   4 |               54% |                       99.973 |
|               80% |                   6 |               54% |                     99.99956 |
|               80% |                   8 |               54% |                   99.9999929 |
|               80% |                  10 |               54% |                  99.99999988 |
|               80% |                  15 |               54% |             99.9999999999961 |
|               80% |                  20 |               54% |         99.99999999999999986 |
|               80% |                  25 |               54% |    99.9999999999999999999955 |
|               85% |                0.01 |               57% |                        1.919 |
|               85% |                 0.1 |               57% |                        17.61 |
|               85% |                   1 |               57% |                        85.60 |
|               85% |                   2 |               57% |                       97.926 |
|               85% |                   3 |               57% |                        99.70 |
|               85% |                   4 |               57% |                       99.957 |
|               85% |                   6 |               57% |                     99.99910 |
|               85% |                   8 |               57% |                    99.999981 |
|               85% |                  10 |               57% |                  99.99999961 |
|               85% |                  15 |               57% |              99.999999999976 |
|               85% |                  20 |               57% |          99.9999999999999985 |
|               85% |                  25 |               57% |     99.999999999999999999909 |

More relativity notebooks are [available here](https://github.com/lookbusy1344/Relativity/blob/main/Python/README.md)

You can read about antimatter rockets in more detail on [Wikipedia](https://en.wikipedia.org/wiki/Antimatter_rocket).